In [1]:
import numpy as np
import os
import time
import tensorflow as tf

In [2]:
path = "C:/Users//Vybhav Shetty//Desktop//Pal.txt"     #download a friend's whatsapp chat
text = open(path, 'rb').read().decode(encoding='utf-8')


In [3]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))
import string


339 unique characters


In [4]:
text1=""
for j,i in enumerate(text):
    if text[j]=='U' and text[j+1]=='s' and text[j+2]=='e' and text[j+3]=='l' and text[j+4]=='e' and text[j+5]=='s' and text[j+6]=='s' and text[j+7]==':':
        for u,v in enumerate(text,start=j+8):
            if text[u].isnumeric() and (text[u+1].isnumeric or text[u+1]=='/') and (text[u+2].isnumeric or text[u+2]=='/') and (text[u+3].isnumeric or text[u+3]=='/'):
                break
            text1=text1+text[u]


In [65]:
#print((text1))

In [6]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text1])

In [7]:

print ('{} ---- characters mapped to int ---- > {}'.format(repr(text1[:13]), text_as_int[:13]))

' See? I use t' ---- characters mapped to int ---- > [ 1 51 68 68 31  1 41  1 84 82 68  1 83]


In [8]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

 
S
e
e
?


In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

' See? I use this\n Also, give me one hour.\n What the hell is that dp😂😂😂\n 😂😂😂😂😂😂😂😂\n Savage\n 😂😂😂😂\n 😂😂😂😂😂'
'😂\n Those were very clever though 😂😂😂👏🏻👏🏻👏🏻👏🏻\n Haan\n 😂😂😂😂\n I am so sleepy\n RISE AND SHINE AMIGO\n oh wa'
"it, you were already awake...\n Your dp is scaring me😂\n Heeyyy!!!!😂😂😂😂👏🏻👏🏻👏🏻\n I'm going to straighten "
"my hair\n No not now, later😂\n Could I tell people that I chopped off your head?\n Because like I'm the "
"queen and you didn't obey me\n For treason\n 😂😂😂😂😂😂😂😂\n How in the world did you come up with that?\n 😂😂😂"


In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  ' See? I use this\n Also, give me one hour.\n What the hell is that dp😂😂😂\n 😂😂😂😂😂😂😂😂\n Savage\n 😂😂😂😂\n 😂😂😂😂'
Target data: 'See? I use this\n Also, give me one hour.\n What the hell is that dp😂😂😂\n 😂😂😂😂😂😂😂😂\n Savage\n 😂😂😂😂\n 😂😂😂😂😂'


In [12]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [13]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [15]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [16]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 339) # (batch_size, sequence_length, vocab_size)


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           86784     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 339)           347475    
Total params: 4,372,563
Trainable params: 4,372,563
Non-trainable params: 0
_________________________________________________________________


In [18]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [19]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 339)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.826226


In [20]:
model.compile(optimizer='adam', loss=loss)

In [21]:

checkpoint_dir = './models/training_Useless_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [22]:
EPOCHS=30

In [ ]:
history = model.fit(dataset, epochs=10, callbacks=[checkpoint_callback])

In [28]:
tf.train.latest_checkpoint(checkpoint_dir)

'./models/training_Useless_checkpoints\\ckpt_10'

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            86784     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 339)            347475    
Total params: 4,372,563
Trainable params: 4,372,563
Non-trainable params: 0
_________________________________________________________________


In [67]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 200

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0
    

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [85]:
print(generate_text(model, start_string=u"what are you doing?"))

what are you doing?
 Let’m still have me up?
 Wtffor my head
 😂😂😂
 That’s not 😂😂
 But there’s something called hiding and eating ren
 But kearing mad boticll?
 Okay
 Yeah
 No attention the office
 NOW YOUR BACK
 😂😂😂
 Wh
